<a href="https://colab.research.google.com/github/mahopman/IEBM-Net/blob/main/intervention_classifier/generate_intervention_classifier_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
local_path = '/content/drive/MyDrive/MS_DataScience/DS595/IEBM-Net_Data'
evidence_integration_path = f'{local_path}/evidence_integration'
pretraining_dataset_path = f'{local_path}/pretraining_dataset'

In [42]:
import json

pmcid2intervention = json.load(open(f'{evidence_integration_path}/pmcid2intervention.json'))
pmcid2content = json.load(open(f'{evidence_integration_path}/evidence_inference/pmc_contents.json'))
sec2label = json.load(open(f'{evidence_integration_path}/evidence_inference/sec2label.json'))
secname2sec = json.load(open(f'{evidence_integration_path}/evidence_inference/secname2sec.json'))

dataset = {}
ctx_id = 0

for pmcid in pmcid2intervention:
    passage = ''
    if pmcid in pmcid2content:
        content = pmcid2content[pmcid]
        for secname, text in content:
            if secname[:len('ABSTRACT')] != 'ABSTRACT': continue
            if sec2label[secname2sec[secname]] == '1':
                passage += text

        dataset[ctx_id] = {}
        dataset[ctx_id]['passage'] = passage
        dataset[ctx_id]['intervention'] = pmcid2intervention[pmcid]
        ctx_id += 1

In [43]:
import glob

pmid2intervention = json.load(open(f'{pretraining_dataset_path}/pmid2intervention.json'))

files = glob.glob(f'{pretraining_dataset_path}/evidence/contexts_*.json')

for file in files:
    data = json.load(open(file))

    for entry in data:
        pmid = entry['pmid']
        if pmid in pmid2intervention:
            dataset[ctx_id] = {}
            dataset[ctx_id]['passage'] = entry['ctx']
            dataset[ctx_id]['intervention'] = pmid2intervention[pmid]
            ctx_id += 1

In [34]:
import numpy as np
n = ctx_id
train_size = int(n * 0.7)
val_size = int(n * 0.15)
test_size = n - train_size - val_size

# Get indices for each section
train_indices = np.random.choice(n, train_size, replace=False)
val_indices = np.random.choice(list(set(range(n)) - set(train_indices)), val_size, replace=False)
test_indices = list(set(range(n)) - set(train_indices) - set(val_indices))

# Split the dataset
train = {k: v for k, v in dataset.items() if k in train_indices}
val = {k: v for k, v in dataset.items() if k in val_indices}
test = {k: v for k, v in dataset.items() if k in test_indices}

In [22]:
with open(f'{local_path}/intervention_classifier/train.json', 'w') as f:
    json.dump(train, f)

with open(f'{local_path}/intervention_classifier/test.json', 'w') as f:
    json.dump(test, f)

with open(f'{local_path}/intervention_classifier/val.json', 'w') as f:
    json.dump(val, f)

{'passage': " We examined the effect of a telemedical coaching (TMC) programme accompanied with or without telemonitoring on weight loss in an occupational healthcare setting with a three-armed randomised controlled trial (NCT01837134 'Pre-results'). Overweight employees (n=104, body mass index [BMI] ≥25\u2009kg/m",
 'intervention': ['DEVICE', 'BEHAVIORAL']}